In [1]:
!ls

Aisha.ipynb                   dataset-original
Design.md                     dataset-original.zip
Nicolas.ipynb                 nara.ipynb
Qianti123.ipynb               plastic-recycling-codes.zip
README.md                     waste-classification-data.zip
__MACOSX


In [ ]:
!unzip -qq dataset-original.zip


In [1]:
!ls

Aisha.ipynb                   dataset-original
Design.md                     dataset-original.zip
Nicolas.ipynb                 nara.ipynb
Qianti123.ipynb               plastic-recycling-codes.zip
README.md                     waste-classification-data.zip
__MACOSX


In [2]:
!rm -rf dataset-original/ __MACOSX/

In [3]:
!ls

Aisha.ipynb                   dataset-original.zip
Design.md                     nara.ipynb
Nicolas.ipynb                 plastic-recycling-codes.zip
Qianti123.ipynb               waste-classification-data.zip
README.md


In [4]:
!ls

Aisha.ipynb                   dataset-original.zip
Design.md                     nara.ipynb
Nicolas.ipynb                 plastic-recycling-codes.zip
Qianti123.ipynb               waste-classification-data.zip
README.md


In [21]:
!unzip -qq waste-classification-data.zip

In [22]:
!ls

Aisha.ipynb                   __MACOSX
DATASET                       dataset-original
Design.md                     dataset-original.zip
Nicolas.ipynb                 nara.ipynb
Qianti.ipynb                  plastic-recycling-codes.zip
README.md                     waste-classification-data.zip


In [23]:
!rm -rf DATASET/

ImportError: cannot import name 'cosine' from 'tensorflow.python.keras.losses' (/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/losses.py)

In [6]:
from google.cloud import vision

In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="AMLI-b8f548e3cf33.json"

In [8]:
client = vision.ImageAnnotatorClient()

In [9]:
!ls

AMLI-b8f548e3cf33.json        dataset-original
Aisha.ipynb                   dataset-original.zip
Design.md                     nara.ipynb
Nicolas.ipynb                 paper_cup.jpg
Qianti123.ipynb               plastic-recycling-codes.zip
README.md                     waste-classification-data.zip


In [24]:
import io

path = 'dataset-original/trash/trash9.jpg'
with io.open(path, 'rb') as image_file:
        content = image_file.read()

# List of Keywords we would be interested in (general): 
### (Note that it is impossible to know every subcategory we should expect)
1. Paper
2. Plastic
3. Glass
4. Cardboard
5. Metal
6. Electronic
7. Trash

In [25]:
image = vision.types.Image(content=content)

In [26]:
response = client.label_detection(image=image)
props = response.image_properties_annotation

In [27]:
response.label_annotations

[mid: "/m/06fvc"
description: "Red"
score: 0.9312219619750977
topicality: 0.9312219619750977
]

In [28]:
response = client.label_detection(image=image)
labels = response.label_annotations
print('Labels:')

for label in labels:
    print(label.description)
    print(label.score)

Labels:
Red
0.9312219619750977


## Now let's try to use the trashnet model and only use images with the complete object

In [2]:
!ls

AMLI-b8f548e3cf33.json        dataset-original
Aisha.ipynb                   dataset-original.zip
Design.md                     nara.ipynb
Nicolas.ipynb                 paper_cup.jpg
Qianti123.ipynb               plastic-recycling-codes.zip
README.md                     waste-classification-data.zip


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.4,
        rotation_range=90,
        horizontal_flip=True,
       # preprocessing_function = preprocess_image
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)#, preprocessing_function = preprocess_image)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        './dataset-original/',  # this is the target directory
        target_size=(100, 100),  # all images will be resized to 100x100
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need categorical labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './hot-dog-not-hot-dog/validation',
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        './hot-dog-not-hot-dog/test',
        target_size=(100, 100),
        batch_size=batch_size, 
        class_mode='categorical')

hello


In [1]:
# Your code goes here
import numpy as np 
import cv2 as cv 

#Use attached zip file (splits data into train, validation, and testing)

In [2]:
def padding(image):
    height = image.shape[0]
    width = image.shape[1]

    left_pad, right_pad, top_pad, bottom_pad = 0, 0, 0, 0
    if height > width:
        left_pad = int((height-width) / 2)
        right_pad = height-width-left_pad
    elif width > height:
        top_pad = int((width-height) / 2)
        bottom_pad = width-height-top_pad

    img_square = cv.copyMakeBorder(
        image,
        top_pad,
        bottom_pad,
        left_pad,
        right_pad,
        cv.BORDER_CONSTANT,
        value=(255,255,255)
    )
    return img_square

def toGray(image):
    # toGray converts RGB values to grayscale values by forming a weighted sum of the R, G, and B components:
    # 0.2989 * R + 0.5870 * G + 0.1140 * B 
    # source: https://www.mathworks.com/help/matlab/ref/rgb2gray.html
    image[:,:,0] = 0.2989*image[:,:,0] +  0.5870*image[:,:,1] + 0.1140*image[:,:,2]
    image[:,:,1] = 0.2989*image[:,:,0] +  0.5870*image[:,:,1] + 0.1140*image[:,:,2]
    image[:,:,2] = 0.2989*image[:,:,0] +  0.5870*image[:,:,1] + 0.1140*image[:,:,2]
    return image

def preprocess_image(image):
    #image =toGray(image)
    image = padding(image)
    image_scaled = cv.resize(image, (300, 300))
    #image_scaled = np.true_divide(image_scaled, 255.0)
    #print(image_scaled.shape)
    return image_scaled

In [3]:
file = 'dataset/paper/paper297.jpg'
image = cv.imread(file)
height = image.shape[0]
width = image.shape[1]

left_pad, right_pad, top_pad, bottom_pad = 0, 0, 0, 0
if height > width:
    left_pad = int((height-width) / 2)
    right_pad = height-width-left_pad
elif width > height:
    top_pad = int((width-height) / 2)
    bottom_pad = width-height-top_pad

img_square = cv.copyMakeBorder(
    image,
    top_pad,
    bottom_pad,
    left_pad,
    right_pad,
    cv.BORDER_CONSTANT,
    value=(255,255,255)
)
print(img_square.shape[0], img_square.shape[1])
new = padding(image)

4032 4032


In [4]:
!pip install opencv-python==3.4.2.17 numpy==1.14.5

     |████████████████████████████████| 42.3MB 7.7MB/s eta 0:00:01     |███████▊                        | 10.3MB 7.8MB/s eta 0:00:05     |████████████████                | 21.2MB 6.9MB/s eta 0:00:04     |████████████████████████████▌   | 37.6MB 1.3MB/s eta 0:00:04
     |████████████████████████████████| 4.7MB 5.9MB/s eta 0:00:01
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [14]:
import os, sys
import glob
import cv2 as cv
root_dir = "./dataset-original/"
count = 0
total = 2527
for filename in glob.iglob(root_dir + '**/*.jpg', recursive=True):
    count += 1
    print(count,'/',total)
    im = cv.imread(filename)
    pad1 = padding(im)
    new = preprocess_image(im)
    cv.imwrite(filename, new)

1 / 2527
2 / 2527
3 / 2527
4 / 2527
5 / 2527
6 / 2527
7 / 2527
8 / 2527
9 / 2527
10 / 2527
11 / 2527
12 / 2527
13 / 2527
14 / 2527
15 / 2527
16 / 2527
17 / 2527
18 / 2527
19 / 2527
20 / 2527
21 / 2527
22 / 2527
23 / 2527
24 / 2527
25 / 2527
26 / 2527
27 / 2527
28 / 2527
29 / 2527
30 / 2527
31 / 2527
32 / 2527
33 / 2527
34 / 2527
35 / 2527
36 / 2527
37 / 2527
38 / 2527
39 / 2527
40 / 2527
41 / 2527
42 / 2527
43 / 2527
44 / 2527
45 / 2527
46 / 2527
47 / 2527
48 / 2527
49 / 2527
50 / 2527
51 / 2527
52 / 2527
53 / 2527
54 / 2527
55 / 2527
56 / 2527
57 / 2527
58 / 2527
59 / 2527
60 / 2527
61 / 2527
62 / 2527
63 / 2527
64 / 2527
65 / 2527
66 / 2527
67 / 2527
68 / 2527
69 / 2527
70 / 2527
71 / 2527
72 / 2527
73 / 2527
74 / 2527
75 / 2527
76 / 2527
77 / 2527
78 / 2527
79 / 2527
80 / 2527
81 / 2527
82 / 2527
83 / 2527
84 / 2527
85 / 2527
86 / 2527
87 / 2527
88 / 2527
89 / 2527
90 / 2527
91 / 2527
92 / 2527
93 / 2527
94 / 2527
95 / 2527
96 / 2527
97 / 2527
98 / 2527
99 / 2527
100 / 2527
101 / 25

757 / 2527
758 / 2527
759 / 2527
760 / 2527
761 / 2527
762 / 2527
763 / 2527
764 / 2527
765 / 2527
766 / 2527
767 / 2527
768 / 2527
769 / 2527
770 / 2527
771 / 2527
772 / 2527
773 / 2527
774 / 2527
775 / 2527
776 / 2527
777 / 2527
778 / 2527
779 / 2527
780 / 2527
781 / 2527
782 / 2527
783 / 2527
784 / 2527
785 / 2527
786 / 2527
787 / 2527
788 / 2527
789 / 2527
790 / 2527
791 / 2527
792 / 2527
793 / 2527
794 / 2527
795 / 2527
796 / 2527
797 / 2527
798 / 2527
799 / 2527
800 / 2527
801 / 2527
802 / 2527
803 / 2527
804 / 2527
805 / 2527
806 / 2527
807 / 2527
808 / 2527
809 / 2527
810 / 2527
811 / 2527
812 / 2527
813 / 2527
814 / 2527
815 / 2527
816 / 2527
817 / 2527
818 / 2527
819 / 2527
820 / 2527
821 / 2527
822 / 2527
823 / 2527
824 / 2527
825 / 2527
826 / 2527
827 / 2527
828 / 2527
829 / 2527
830 / 2527
831 / 2527
832 / 2527
833 / 2527
834 / 2527
835 / 2527
836 / 2527
837 / 2527
838 / 2527
839 / 2527
840 / 2527
841 / 2527
842 / 2527
843 / 2527
844 / 2527
845 / 2527
846 / 2527
847 / 2527

1461 / 2527
1462 / 2527
1463 / 2527
1464 / 2527
1465 / 2527
1466 / 2527
1467 / 2527
1468 / 2527
1469 / 2527
1470 / 2527
1471 / 2527
1472 / 2527
1473 / 2527
1474 / 2527
1475 / 2527
1476 / 2527
1477 / 2527
1478 / 2527
1479 / 2527
1480 / 2527
1481 / 2527
1482 / 2527
1483 / 2527
1484 / 2527
1485 / 2527
1486 / 2527
1487 / 2527
1488 / 2527
1489 / 2527
1490 / 2527
1491 / 2527
1492 / 2527
1493 / 2527
1494 / 2527
1495 / 2527
1496 / 2527
1497 / 2527
1498 / 2527
1499 / 2527
1500 / 2527
1501 / 2527
1502 / 2527
1503 / 2527
1504 / 2527
1505 / 2527
1506 / 2527
1507 / 2527
1508 / 2527
1509 / 2527
1510 / 2527
1511 / 2527
1512 / 2527
1513 / 2527
1514 / 2527
1515 / 2527
1516 / 2527
1517 / 2527
1518 / 2527
1519 / 2527
1520 / 2527
1521 / 2527
1522 / 2527
1523 / 2527
1524 / 2527
1525 / 2527
1526 / 2527
1527 / 2527
1528 / 2527
1529 / 2527
1530 / 2527
1531 / 2527
1532 / 2527
1533 / 2527
1534 / 2527
1535 / 2527
1536 / 2527
1537 / 2527
1538 / 2527
1539 / 2527
1540 / 2527
1541 / 2527
1542 / 2527
1543 / 2527
1544

2145 / 2527
2146 / 2527
2147 / 2527
2148 / 2527
2149 / 2527
2150 / 2527
2151 / 2527
2152 / 2527
2153 / 2527
2154 / 2527
2155 / 2527
2156 / 2527
2157 / 2527
2158 / 2527
2159 / 2527
2160 / 2527
2161 / 2527
2162 / 2527
2163 / 2527
2164 / 2527
2165 / 2527
2166 / 2527
2167 / 2527
2168 / 2527
2169 / 2527
2170 / 2527
2171 / 2527
2172 / 2527
2173 / 2527
2174 / 2527
2175 / 2527
2176 / 2527
2177 / 2527
2178 / 2527
2179 / 2527
2180 / 2527
2181 / 2527
2182 / 2527
2183 / 2527
2184 / 2527
2185 / 2527
2186 / 2527
2187 / 2527
2188 / 2527
2189 / 2527
2190 / 2527
2191 / 2527
2192 / 2527
2193 / 2527
2194 / 2527
2195 / 2527
2196 / 2527
2197 / 2527
2198 / 2527
2199 / 2527
2200 / 2527
2201 / 2527
2202 / 2527
2203 / 2527
2204 / 2527
2205 / 2527
2206 / 2527
2207 / 2527
2208 / 2527
2209 / 2527
2210 / 2527
2211 / 2527
2212 / 2527
2213 / 2527
2214 / 2527
2215 / 2527
2216 / 2527
2217 / 2527
2218 / 2527
2219 / 2527
2220 / 2527
2221 / 2527
2222 / 2527
2223 / 2527
2224 / 2527
2225 / 2527
2226 / 2527
2227 / 2527
2228

In [15]:
## helper functions ##
import random
import shutil
## splits indices for a folder into train, validation, and test indices with random sampling
    ## input: folder path
    ## output: train, valid, and test indices    
def split_indices(folder,seed1,seed2):    
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    ## train indices
    random.seed(seed1)
    train = random.sample(list(range(1,n+1)),int(.5*n))

    ## temp
    remain = list(set(full_set)-set(train))

    ## separate remaining into validation and test
    random.seed(seed2)
    valid = random.sample(remain,int(.5*len(remain)))
    test = list(set(remain)-set(valid))
    
    return(train,valid,test)

## gets file names for a particular type of trash, given indices
    ## input: waste category and indices
    ## output: file names 
def get_names(waste_type,indices):
    file_names = [waste_type+str(i)+".jpg" for i in indices]
    return(file_names)    

## moves group of source files to another folder
    ## input: list of source files and destination folder
    ## no output
def move_files(source_files,destination_folder):
    for file in source_files:
        shutil.move(file,destination_folder)

In [16]:
# paths will be train/cardboard, train/glass, etc...
subsets = ['train','valid', 'test']
waste_types = ['cardboard', 'glass', 'metal','paper', 'plastic', 'trash']

## create destination folders for data subset and waste type
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data',subset,waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)
            
# if not os.path.exists(os.path.join('data1','test')):
#     os.makedirs(os.path.join('data1','test'))
            
## move files to destination folders for each waste type
for waste_type in waste_types:
    source_folder = os.path.join('dataset-original',waste_type)
    train_ind, valid_ind, test_ind = split_indices(source_folder,1,1)
    try:
        ## move source files to train
        train_names = get_names(waste_type,train_ind)
        train_source_files = [os.path.join(source_folder,name) for name in train_names]
        train_dest = "data/train/"+waste_type
        move_files(train_source_files,train_dest)

        ## move source files to valid
        valid_names = get_names(waste_type,valid_ind)
        valid_source_files = [os.path.join(source_folder,name) for name in valid_names]
        valid_dest = "data/valid/"+waste_type
        move_files(valid_source_files,valid_dest)

        ## move source files to test
        test_names = get_names(waste_type,test_ind)
        test_source_files = [os.path.join(source_folder,name) for name in test_names]
        ## I use data/test here because the images can be mixed up
        test_dest = "data/test/"+waste_type
        move_files(test_source_files,test_dest)
    except:
        print('error')
        pass

error
error
error
error
error


In [128]:
!ls

AMLI-b8f548e3cf33.json        dataset-original
Aisha.ipynb                   dataset-original.zip
Design.md                     nara.ipynb
Nicolas.ipynb                 paper_cup.jpg
Qianti123.ipynb               plastic-recycling-codes.zip
README.md                     waste-classification-data.zip
data


In [18]:
import tensorflow as tf
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.4,
        rotation_range=90,
        horizontal_flip=True,
        #preprocessing_function = preprocess_image
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)#, preprocessing_function = preprocess_image)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        './data/train',  # this is the target directory
       # target_size=(100, 100),  # all images will be resized to 100x100
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need categorical labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './data/valid',
        #target_size=(100, 100),
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        './data/test',
        #target_size=(100, 100),
        batch_size=batch_size, 
        class_mode='categorical')

Found 1264 images belonging to 6 classes.
Found 632 images belonging to 6 classes.
Found 631 images belonging to 6 classes.


In [1]:
from classification_models.resnet import ResNet18, preprocess_input

Using TensorFlow backend.
